# Import dependencies

In [32]:
import os
import re
import unicodedata
import spacy
import pandas as pd
from matplotlib import pyplot as plt
from dotenv import load_dotenv

# Load dataset

In [33]:
# load the environment variables from the .env file
load_dotenv()
# get the dataset path from the environment variables
dataset_path = os.environ.get("DATASET_PATH")
# get the articles path for the known publisher
articles_path = os.path.join(dataset_path, "articles")
# load the dataset into the notebook
df = pd.read_csv(f"{dataset_path}/article_info_V2.csv", index_col=0, parse_dates=[1])

# Data cleaning

## Define cleaning functions

In [34]:
def clean_string(str):
    # make the string lowercase
    str = str.lower()
    # remove all non-alphanumeric characters
    str = re.sub(r"[^\w\s]", "", str)
    # remove the leading and trailing spaces
    return str.strip()

def parse_list_from_string(str):
    # split the string on each comma
    raw_list = str.split(",")
    # clean every string in the list
    return list(map(clean_string, raw_list))

## Drop unused columns

In [35]:
# drop the author, type and keywords columns
df.drop(columns=["Author", "Type", "Keywords"], inplace=True)

## Drop empty rows

In [36]:
# remove rows with empty date values
df.dropna(subset=["Date"], inplace=True)

## Merge datasets

In [37]:
# read the external dataset
external_df = pd.read_csv(f"{dataset_path}/other-articles.csv", index_col=0, parse_dates=[1], sep=";")
# remove the url column from the external dataset
external_df.drop(columns=["Url"], inplace=True)
# add the external dataset to the main dataset
df = pd.concat([df, external_df])
# reset the index of the main dataset
df.reset_index(drop=True, inplace=True)
# clear the external dataset variable
del external_df

## Remove duplicate articles

In [38]:
# count the number of entries before duplicate removal
entries_before_duplicate_removal = len(df)
# remove duplicate entries by title and date
df.drop_duplicates(subset=["Title", "Date"], keep="first", inplace=True)
# count the number of entries after duplicate removal
entries_after_duplicate_removal = len(df)
# print removed duplicate count
print(f"Removed {entries_before_duplicate_removal - entries_after_duplicate_removal} duplicate entries.")

Removed 3 duplicate entries.


## Parse the string lists

In [39]:
# show the tags column before parsing
print(f"The type of values in the Tags column is {type(df.loc[0, 'Tags'])}.")
# convert the raw string values of the Tags column to lists of strings
df["Tags"] = df["Tags"].apply(parse_list_from_string)
# show the tags column after parsing
print(f"The type of values in the Tags column is {type(df.loc[0, 'Tags'])}.")

The type of values in the Tags column is <class 'str'>.
The type of values in the Tags column is <class 'list'>.


## Define article loading function

In [40]:
# load article by title
def load_article(title, load_contents=True):
    # create file name from title
    file_name = f"{title}.txt"
    # get the path of the article
    file_path = os.path.join(articles_path, file_name)
    # read the article
    file = open(file_path, "r", encoding="utf-8")
    # return the contents of the article if requested
    if load_contents:
        # read the contents of the article
        contents = file.read()
        # close the file
        file.close()
        # return the contents of the article
        return contents
    # close the file
    file.close()
    # return the article path
    return file

## Remove articles which cannot be found

In [41]:
# initialize error count to 0 
err_count = 0

# iterate over dataset with index
for index, row in df.iterrows():
    # get the title of the article
    title = row["Title"]
    try:
        # attempt to load the article
        article = load_article(title)
    except:
        # if the article cannot be loaded, increment the error count
        err_count += 1
        # remove row from dataset
        df.drop(index, inplace=True)
        # continue to next row
        continue

# print the number of errors
print(f"{err_count} files could not be loaded by title!")

42 files could not be loaded by title!


## Define article cleaning methods

In [42]:
# # replace accented characters with their unaccented counterparts
# def remove_accented_characters(text):
#     return unicodedata.normalize("NFKD", text).encode("ascii", "ignore").decode("utf-8", "ignore")

In [61]:
# mainly based on https://medium.com/analytics-vidhya/text-preprocessing-nlp-basics-430d54016048

# load spacy model
nlp = spacy.load("en_core_web_sm")

# remove any newline characters
def remove_newlines(text):
    # remove all newline characters
    text = re.sub(r"\n\n", " ", text)
    text = re.sub(r"\n", " ", text)
    text = re.sub(r"\r\r", " ", text)
    text = re.sub(r"\r", " ", text)
    return text

# remove double whitespace characters
def remove_double_whitespace(text):
    return re.sub(r"\s\s+", " ", text)

# replace shortened grammar with full grammar
def replace_grammar(text):
    # replace the grammar
    text = text.replace("\it's", "it is")
    text = re.sub(r"it's", "it is", text)
    # TODO: NOT DONE YET, REPLACE THIS!

    text = re.sub(r"'s", "its", text)

    text = text.replace("\'ve", " have")
    text = text.replace("n\'t", " not")
    text = text.replace("\'re", " are")
    text = text.replace("\'d", " would")
    text = text.replace("\'ll", " will")
    text = text.replace("\'m", " am")
    # return the text
    return text

# remove noise from text
def remove_noise(text):
    # remove newline characters
    text = remove_newlines(text)
    # replace short grammar with full grammar
    text = replace_grammar(text)
    # remove punctuation
    text = re.sub(r"[^\w\s]", "", text)
    # remove digits
    text = re.sub(r"\d+", "", text)
    # remove double whitespace characters
    text = remove_double_whitespace(text)
    # return the cleaned text
    return text

def tokenize(text):
    # tokenize the text
    tokens = nlp(text)
    # return the tokens
    return tokens

def remove_stop_words(tokens):
    # remove stop words
    return [token for token in tokens if not token.is_stop]

def lemmatize(tokens):
    # lemmatize the text
    return [token.lemma_ for token in tokens]

def lowercase(tokens):
    # lowercase the tokens
    return [token.lower() for token in tokens]

def preprocess_text(text):
    text = remove_noise(text)
    tokens = tokenize(text)
    tokens = remove_stop_words(tokens)
    tokens = lemmatize(tokens)
    # return lowercase(tokens)
    return tokens


In [62]:
article_contents = load_article(df["Title"][1])

tokens = preprocess_text(article_contents)

print(df["Tags"][1])
print(tokens)

['contraband', 'zetas', 'haiti', 'cocaine', 'el salvador', 'bolivia', 'judicial reform', 'elites and crime']
['explosive', 'new', 'report', 'suggest', 'highprofile', 'assassination', 'Jovenel', 'Moïse', 'relate', 'crackdown', 'drug', 'trafficking', 'list', 'compile', 'haitian', 'business', 'political', 'elite', 'involve', 'trade', 'add', 'theory', 'possible', 'motive', 'president', 'kill', 'shoot', 'dead', 'President', 'Moïse', 'plan', 'hand', 'name', 'government', 'accord', 'New', 'York', 'Times', 'report', 'publish', 'December', 'Times', 'speak', 'senior', 'haitian', 'adviser', 'official', 'knowledge', 'document', 'unnamed', 'official', 'tell', 'Times', 'hitman', 'confess', 'ransack', 'Moïse', 'house', 'search', 'list', 'president', 'order', 'official', 'spare', 'power', 'broker', 'help', 'propel', 'office', 'Times', 'report', 'central', 'figure', 'include', 'list', 'accord', 'Times', 'businessman', 'Charles', 'SaintRémy', 'alia', 'Kiko', 'Times', 'previously', 'report', 'antidrug', 

# Data preparation

## Define one hot encoding function

In [ ]:
# get unique values from a 2D array of strings
def get_unique_value_frequency(df_column):
    # create a dictionary to store the unique values
    unique_values = {}
    # iterate over the column
    for value_list in df_column:
        # iterate over the values in the list
        for value in value_list:
            if value not in unique_values:
                # if the value is not in the dictionary, add it
                unique_values[value] = 1
            else:
                # if the value is in the dictionary, increment the value
                unique_values[value] += 1
    # return the dictionary of unique values
    return unique_values

# check if a list contains a certain word and returns a binary boolean value
def list_has_word(l, word):
    return word in l and 1 or 0

# one hot encode a dataframe's column that contains lists of strings in each value
def custom_one_hot_encoding(df, column_name, prefix=None, prefix_sep="_"):
    # create a dictionary to store the one hot encoded columns
    one_hot_encoded_columns = {}
    # get the unique values from the column
    unique_values = get_unique_value_frequency(df[column_name])
    # iterate over the unique values
    for unique_value in unique_values:
        # create a clean string of the unique value
        clean_unique_value = unique_value.replace(" ", "_")
        # create a new column name
        new_column_name = prefix and f"{prefix}{prefix_sep}{clean_unique_value}" or f"{column_name}{prefix_sep}{clean_unique_value}"
        # one hot encode the column using the current unique value
        ohe_list = df[column_name].apply(lambda l: list_has_word(l, unique_value))
        # add the new list to the dictionary
        one_hot_encoded_columns[new_column_name] = ohe_list
    # return a new dataframe with the one hot encoded columns
    return pd.DataFrame(one_hot_encoded_columns)

## Execute one hot encoding function

In [ ]:
# one hot encode the tags column of the dataframe
ohe_tags_df = custom_one_hot_encoding(df, "Tags", "tag")
# merge the one hot encoded tags dataframe with the main dataframe by index
df = df.join(ohe_tags_df)
# drop the tags column from the main dataframe
print("CHANGE THIS LATER!!!!")
# df.drop(columns=["Tags"], inplace=True)
# delete the one hot encoded dataframe variable
del ohe_tags_df